## Initialization

In [40]:
import os
import openai
import sys
import re
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## Articles loader

In [78]:
from langchain.document_loaders import PyPDFLoader
loaders = [
    PyPDFLoader("./articles/0dacb0b566cb38438b84aa092176c6d42bf4402d(16).pdf")
]

articles = []
for loader in loaders:
    article = loader.load()
    articles.append(article)

In [79]:
len(articles[0])

7

## Articles Splitting

In [80]:
text = "\n".join([page.page_content for page in articles[0]])

In [81]:
headers = [
    r"^\s*Abstract\s*$",
    r"^\s*Introduction\s*$",
    r"^\s*Methods\s*$",
    r"^\s*Methodology\s*$",
    r"^\s*Results\s*$",
    r"^\s*Discussion\s*$",
    r"^\s*Conclusion\s*$",
    r"^\s*References\s*$"
]

headers_to_split_on = [
    ("#", "Abstract"),
    ("#", "Introduction"),
    ("#", "Methods"),
    ("#", "Methodology"),
    ("#", "Results"),
    ("#", "Discussion"),
    ("#", "Conclusion"),
    ("#", "References"),
]

In [82]:
def convert_to_markdown(text, headers):
    lines = text.split("\n")
    markdown_text = []
    
    for line in lines:
        header_found = False
        for header in headers:
            if re.match(header, line.strip(), re.IGNORECASE):
                markdown_text.append(f"# {line.strip()}")
                header_found = True
                break
        if not header_found:
            markdown_text.append(line)
    
    return "\n".join(markdown_text)

# Convertendo o texto para formato Markdown
markdown_text = convert_to_markdown(text, headers)

In [83]:
# Dividindo o texto Markdown usando MarkdownHeaderTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_text)

In [85]:
size = len(md_header_splits)
md_header_splits[1]

print(size)

5


In [38]:
# TODO: implement small sections

## Vectorstores and Embedding